In [1]:
# !pip install snowflake-connector-python
# !pip install snowflake-sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import gensim
import re
from gensim.models.phrases import Phrases, Phraser
import numpy as np

ModuleNotFoundError: No module named 'sqlalchemy'

In [2]:
ENGINE = create_engine(
    'snowflake://{user}:{password}@{account}/'.format(
        user='eric.clausenbrown@offerupnow.com',
        password='1GreatDay!',
        account='offerup',
    ),
)

NameError: name 'create_engine' is not defined

In [ ]:
sql2 = """
with myviews_raw_0 as (
select s.header_user_id, date_trunc(day, s.header_timestamp) as timebin, 
  array_to_string(split(substring(trim(lower(s.query)),0,30), ' '), '_') as concat_keyword_query, 
  substring(trim(lower(s.query)),0,30) as keyword_query
  from events_prod.public.materialized_search_performed_event s
  where s.results_requested_start_index=0
  and s.header_timestamp > '2018-06-15'
  and s.header_timestamp < '2018-07-05'
  and length(s.query)>0
  and s.header_user_id != 0
  //group by iv.header_user_id, hour_ts having count(distinct iv.item_id) >= 20 and count(distinct iv.item_id) <= 100
),
myviews_raw as (
  select distinct header_user_id, timebin, concat_keyword_query, keyword_query
  from myviews_raw_0
),
myviews as (
select header_user_id, timebin
  from myviews_raw
  group by header_user_id, timebin having count(distinct concat_keyword_query) >= 5 and count(distinct concat_keyword_query) <= 50
)
select mr.header_user_id, mr.timebin, listagg(mr.keyword_query, ' ') as query_cat, listagg(concat_keyword_query, ' ') as whole_query_cat,
  count(*) as nitems
  from myviews_raw mr
  join myviews m on m.header_user_id = mr.header_user_id
  and m.timebin = mr.timebin
  group by mr.header_user_id, mr.timebin
  order by mr.header_user_id, mr.timebin;
"""

In [ ]:
def run_query(sql, engine=ENGINE):
    print(sql)
    try:
        engine.execute("use warehouse analytics_wh;")
        #engine.execute("use database analytics;")
        #engine.execute("use schema ebay_scam;")
        engine.execute("use role data_science;")
        df = pd.read_sql(sql, engine)
    finally:
        engine.dispose()
    return df

In [ ]:
df = run_query(sql2)

In [ ]:
class Item2Vec(object):
    
    def __init__(self, documents, token_pattern=r"(?u)\b\w+\b"):
        self.tokenizer = re.compile(token_pattern)
        documents0 = [self.tokenizer.findall(x.lower()) for x in documents]
        phrases = Phrases(iter(documents0), min_count=5, threshold=10)  # train phrase model
        self.bigrams = Phraser(phrases)
        documents = [self.bigrams[x] for x in documents0]
        #iter_obj = self.inter_obj(docs)
        
        self.model = gensim.models.Word2Vec(documents, min_count=10, sg=0, iter=50)
        
#     def inter_obj(self, docs):
#         for doc in docs:
#             yield doc
            
            
#     def item_vector(self, mystr):
#         words = self.tokenizer.findall(mystr.lower())
#         word_phrases = self.bigrams[words]
#         ret = np.zeros(self.model.vector_size)
#         n = 0
#         for phrase in word_phrases[1:]:
#             try:
#                 ret = ret + self.model.wv[phrase]
#                 n += 1
#             except:
#                 continue
#         return ret/float(n)
    
    def predict(self, text):
       # def item_vector(mystr, w2v):
        words = self.tokenizer.findall(text.lower())
        word_phrases = self.bigrams[words]
        ret = np.zeros(self.model.vector_size)
        n = 0
        for phrase in word_phrases:
            try:
                ret = ret + self.model.wv[phrase]
                n += 1
            except:
                continue
        return ret/float(n)
        

In [ ]:
titles = df.query_cat.values

In [ ]:
item2vec = Item2Vec(documents=titles)

In [ ]:
item2vec.predict("iphone 6s")